In [10]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd

# Load environment variables from .env
load_dotenv()

# Fetch variables
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")


connection = psycopg2.connect(
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT,
    dbname=DBNAME
)
print("Connection successful!")

# Create a cursor to execute SQL queries
cursor = connection.cursor()

# Example query
cursor.execute("SELECT NOW();")
result = cursor.fetchone()
print(f"Current Time: {result}", )





Connection successful!
Current Time: (datetime.datetime(2025, 9, 29, 13, 6, 12, 668068, tzinfo=datetime.timezone.utc),)


In [11]:
# Query to fetch transcripts (exclude NULL transcripts)
query = "SELECT transcript FROM __youtube_transcripts WHERE transcript IS NOT NULL LIMIT 100;"
cursor.execute(query)


In [14]:
# SQL query to estimate tokens
query = """
SELECT
    COUNT(*) AS num_transcripts,
    COALESCE(SUM(ARRAY_LENGTH(STRING_TO_ARRAY(transcript, ' '), 1)), 0) AS total_words,
    COALESCE(SUM(ARRAY_LENGTH(STRING_TO_ARRAY(transcript, ' '), 1)) * 1.3, 0) AS estimated_tokens,
    COALESCE(SUM(LENGTH(transcript)) / 1024.0 / 1024.0, 0) AS total_mb
FROM __youtube_transcripts
WHERE transcript IS NOT NULL AND transcript != '';
"""
cursor.execute(query)

result = cursor.fetchone()

# Parse results
num_transcripts, total_words, estimated_tokens, total_mb = result
print(f"Number of transcripts: {num_transcripts}")
print(f"Total words: {total_words}")
print(f"Estimated BERT tokens: {estimated_tokens:.2f}")
print(f"Total size: {total_mb:.2f} MB")

Number of transcripts: 499
Total words: 517796
Estimated BERT tokens: 673134.80
Total size: 3.01 MB


In [ ]:
# Verify table and column exist
cursor.execute("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema = 'public' AND table_name = '__youtube_transcripts' AND column_name = 'transcript';
""")
if not cursor.fetchone():
    raise Exception("Table '__youtube_transcripts' or column 'transcript' not found.")